# Script to write sql queries on ksqldb
- Import the necessary Dependencies

In [1]:
import os
import json
from requests import Session

- Initialize ksqlDB client session

In [2]:
ksql_client = Session()
ksql_client.headers.update({'Content-Type': 'application/vnd.ksql.v1+json'})

- Set the KSQLDB-Server URL

In [15]:
server_url = "http://svc-kafka-ksqldb-server-headless.starlake-kafka:8088"

- set the folders containing the sql queries

In [16]:
stream_folder = "sql_queries_stream"
table_folder = "sql_queries_table"
combined_folder = "sql_queries_combined"
all_folder = "all_stream_combined"

- Function that interates the over the folder and execute the sql commands on ksqldb

In [17]:
def create_ksql_queries(sql_folder):
    for filename in os.listdir(sql_folder):
        if filename.endswith(".sql"):
            file_path = os.path.join(sql_folder, filename)
            
            # Read SQL command from file
            with open(file_path, "r") as file:
                sql_command = file.read()

            # Execute SQL command
            response = ksql_client.post(f"{server_url}/ksql", json={'ksql': sql_command})
            
            # Check response status code
            response.raise_for_status()

            # Print response
            print(f"Response for {filename}: {response.json()}")

In [18]:
create_ksql_queries(stream_folder)

Response for stream_1.sql: [{'@type': 'currentStatus', 'statementText': "CREATE STREAM SHEEP_STREAM_1 (ID INTEGER, FARM STRING, SHEEP_ID INTEGER, LONGITUDE DOUBLE, LATITUDE DOUBLE, DATE_TIME STRING, FILE_PATH STRING) WITH (CLEANUP_POLICY='delete', KAFKA_TOPIC='sheeps', KEY_FORMAT='KAFKA', VALUE_FORMAT='AVRO');", 'commandId': 'stream/`SHEEP_STREAM_1`/create', 'commandStatus': {'status': 'SUCCESS', 'message': 'Stream created', 'queryId': None}, 'commandSequenceNumber': 2, 'warnings': []}]
Response for stream_2.sql: [{'@type': 'currentStatus', 'statementText': "CREATE STREAM WHALE_STREAM_2 (ID INTEGER, FARM STRING, WHALE_ID INTEGER, LONGITUDE DOUBLE, LATITUDE DOUBLE, DATE_TIME STRING, FILE_PATH STRING) WITH (CLEANUP_POLICY='delete', KAFKA_TOPIC='whales', KEY_FORMAT='KAFKA', VALUE_FORMAT='AVRO');", 'commandId': 'stream/`WHALE_STREAM_2`/create', 'commandStatus': {'status': 'SUCCESS', 'message': 'Stream created', 'queryId': None}, 'commandSequenceNumber': 4, 'warnings': []}]
Response for str

In [19]:
create_ksql_queries(table_folder)

Response for bird_name3_table.sql: [{'@type': 'currentStatus', 'statementText': "CREATE TABLE BIRD_NAME_3 (BIRD_ID INTEGER PRIMARY KEY, BIRD_NAME STRING) WITH (CLEANUP_POLICY='compact', KAFKA_TOPIC='bird_names_farm', KEY_FORMAT='DELIMITED', VALUE_FORMAT='DELIMITED');", 'commandId': 'table/`BIRD_NAME_3`/create', 'commandStatus': {'status': 'SUCCESS', 'message': 'Table created', 'queryId': None}, 'commandSequenceNumber': 8, 'warnings': []}]
Response for sheep_name1_table.sql: [{'@type': 'currentStatus', 'statementText': "CREATE TABLE SHEEP_NAME_1 (SHEEP_ID INTEGER PRIMARY KEY, SHEEP_NAME STRING) WITH (CLEANUP_POLICY='compact', KAFKA_TOPIC='sheep_names_farm', KEY_FORMAT='DELIMITED', VALUE_FORMAT='DELIMITED');", 'commandId': 'table/`SHEEP_NAME_1`/create', 'commandStatus': {'status': 'SUCCESS', 'message': 'Table created', 'queryId': None}, 'commandSequenceNumber': 10, 'warnings': []}]
Response for whale_name2_table.sql: [{'@type': 'currentStatus', 'statementText': "CREATE TABLE WHALE_NAME_2

In [20]:
create_ksql_queries(combined_folder)

Response for bird_combined.sql: [{'@type': 'currentStatus', 'statementText': "CREATE STREAM BIRD_COMBINED WITH (CLEANUP_POLICY='delete', KAFKA_TOPIC='BIRD_COMBINED', PARTITIONS=1, REPLICAS=1, RETENTION_MS=86400000) AS SELECT\n  BIRD_STREAM_3.ID ID,\n  BIRD_STREAM_3.FARM FARM,\n  BIRD_STREAM_3.BIRD_ID BIRD_STREAM_3_BIRD_ID,\n  BIRD_STREAM_3.LONGITUDE LONGITUDE,\n  BIRD_STREAM_3.LATITUDE LATITUDE,\n  BIRD_STREAM_3.DATE_TIME DATE_TIME,\n  BIRD_STREAM_3.FILE_PATH FILE_PATH,\n  BIRD_NAME_3.BIRD_ID BIRD_ID,\n  BIRD_NAME_3.BIRD_NAME BIRD_NAME\nFROM BIRD_STREAM_3 BIRD_STREAM_3\nINNER JOIN BIRD_NAME_3 BIRD_NAME_3 ON ((BIRD_STREAM_3.BIRD_ID = BIRD_NAME_3.BIRD_ID))\nEMIT CHANGES;", 'commandId': 'stream/`BIRD_COMBINED`/create', 'commandStatus': {'status': 'SUCCESS', 'message': 'Created query with ID CSAS_BIRD_COMBINED_13', 'queryId': 'CSAS_BIRD_COMBINED_13'}, 'commandSequenceNumber': 14, 'warnings': []}]
Response for sheep_combined.sql: [{'@type': 'currentStatus', 'statementText': "CREATE STREAM S

In [21]:
create_ksql_queries(all_folder)

Response for all_stream.sql: [{'@type': 'currentStatus', 'statementText': "CREATE STREAM ALL_STREAM WITH (CLEANUP_POLICY='delete', KAFKA_TOPIC='ALL_STREAM', PARTITIONS=1, REPLICAS=1, RETENTION_MS=86400000) AS SELECT\n  SHEEP_COMBINED.SHEEP_STREAM_1_SHEEP_ID ANIMAL_ID_KEY,\n  SHEEP_COMBINED.SHEEP_ID ANIMAL_ID,\n  SHEEP_COMBINED.SHEEP_NAME ANIMAL_NAME,\n  SHEEP_COMBINED.FARM FARM,\n  SHEEP_COMBINED.LONGITUDE LONGITUDE,\n  SHEEP_COMBINED.LATITUDE LATITUDE,\n  SHEEP_COMBINED.DATE_TIME DATE_TIME,\n  SHEEP_COMBINED.FILE_PATH FILE_PATH\nFROM SHEEP_COMBINED SHEEP_COMBINED\nEMIT CHANGES;", 'commandId': 'stream/`ALL_STREAM`/create', 'commandStatus': {'status': 'SUCCESS', 'message': 'Created query with ID CSAS_ALL_STREAM_19', 'queryId': 'CSAS_ALL_STREAM_19'}, 'commandSequenceNumber': 20, 'warnings': []}, {'@type': 'currentStatus', 'statementText': 'INSERT INTO ALL_STREAM SELECT\n  WHALE_STREAM_2_WHALE_ID ANIMAL_ID_KEY,\n  WHALE_ID ANIMAL_ID, \n  WHALE_NAME ANIMAL_NAME,\n  FARM,\n  LONGITUDE,\n  L